In [1]:
import pickle
import time
import re
import os
import copy
import pandas as pd
from multiprocessing import Process, Queue
from IPython.display import clear_output as clear

def readDataFiles_(path):
    readList = []
    with open(path, 'rb') as f:
        readList = pickle.load(f)
    return readList


def saveDataFiles_(data, path):
    with open(path, 'wb') as f:
        readList = pickle.dump(data, f)

In [20]:
# from processing.tagsHandle import readDataFiles_

r1 = []
r2 = []
r3 = []
r4 = []

r1 = list(set(readDataFiles_('./data/pkl/newPKL/r1_province_1229.pkl')))
r1_short = list(set(readDataFiles_('./data/pkl/newPKL/r1_provinceShort_1229.pkl')))
# r1.extend(r1_short)
r2 = list(set(readDataFiles_('./data/pkl/newPKL/r2_city_1229.pkl')))
r2_short = list(set(readDataFiles_('./data/pkl/newPKL/r2_cityshort_1229.pkl')))
# r2.extend(r2_short)
r3 = list(set(readDataFiles_('./data/pkl/newPKL//r3_town_last_1229.pkl')))
r3_short = list(set(readDataFiles_('./data/pkl/newPKL/r3_townShort_1229.pkl')))
# r3.extend(r3_short)
r4 = list(set(readDataFiles_('./data/pkl/newPKL/r4_country_last_1229.pkl')))
r4_short = list(set(readDataFiles_('./data/pkl/newPKL//r4_countryShort_last_1229.pkl')))
# r4.extend(r4_short)
# r5 = readDataFiles_('/home/bangsun/research/nlpAddress/data/generate/pkl/r5_village2_1015.pkl')

r1_r4_words_labels = {}
r1_r4_words = []
repeat_w = []
for i, r in enumerate([r1, r2, r3, r4, r1_short]):
    for w in r:
        if w in r1_r4_words_labels:
            print(w, int(i%4) + 1, r1_r4_words_labels[w])
            repeat_w.append((w, int(i%4) + 1, r1_r4_words_labels[w]))
            continue
        r1_r4_words_labels[w] = "R" + str(int(i%4) + 1)
    r1_r4_words.extend(r)

白塔区 4 R3
加格达奇区 4 R3


In [26]:
r1_r4_words.remove('加格达奇区')

ValueError: list.remove(x): x not in list

In [27]:
all_data = readDataFiles_('data/source/76wdata_r20_r22_r23_r24clear/train_all_data.pkl')

In [28]:
len(all_data)

764336

In [29]:
# 处理两个行政区划问题

In [30]:
def check_words(temp_w, start_r1_r4):
    for r1_r4_w in r1_r4_words:
        if temp_w.startswith(r1_r4_w):
            temp_w = temp_w[len(r1_r4_w):]
            temp_w = check_words(temp_w, start_r1_r4)
            start_r1_r4.insert(0, r1_r4_w)
            break
    return temp_w

def excute_data_fun(q_all_data, q_result):
    while True:
#         time.sleep(0.1)d
        items = q_all_data.get()
        if items == None:
            break
        tag_proplem = 0
        temp = items
        words, labels= items
        new_words = []
        new_labels = []
        for _, w in enumerate(words):
            temp_w = w
            tag = 0
            start_r1_r4 = []
            
            temp_w = check_words(temp_w, start_r1_r4)
            if len(start_r1_r4) >= 2:
                tag_proplem = 1
                if not temp_w:
                    for s_w in start_r1_r4:
                        new_words.append(s_w)
                        new_labels.append(r1_r4_words_labels[s_w])
                else:
                    for s_w in start_r1_r4[:-1]:
                        new_words.append(s_w)
                        new_labels.append(r1_r4_words_labels[s_w])
                        
                    new_words.append(start_r1_r4[-1] + temp_w)
                    new_labels.append(labels[_])
            else:
                new_words.append(w)
                new_labels.append(labels[_])
        q_result.put((tag_proplem, temp, (new_words, new_labels)))
    q_result.put(None)

In [31]:
q_all_data = Queue()
q_result = Queue()
# del q_all_data
# del q_result

In [32]:
process_list = []
for i in range(15):
    process_list.append(Process(target=excute_data_fun, args=(q_all_data, q_result)))

In [33]:
for temp_d in all_data:
    q_all_data.put(temp_d)
for i in range(len(process_list)):
    q_all_data.put(None)

In [34]:
excute_all_data = []
no_prop_data = []
question_data = []
question_data_excute = []

for process in process_list:
    process.start()

i = 0
end_signal = []
while not len(end_signal) == len(process_list):
    res = q_result.get()
    if res == None:
        end_signal.append(res)
    else:
        clear()
        i+=1
        print(i)
        tag_proplem, d, excute_d = res
        if tag_proplem:
            question_data.append(d)
        else:
            no_prop_data.append(d)
        excute_all_data.append(excute_d)
        if tag_proplem:
            question_data_excute.append(excute_d)

764336


In [45]:
question_data_excute

[(['浙江省',
   '杭州市',
   '上城区',
   '清江路',
   '120号',
   '杭州星座女装电子商务楼',
   '8511',
   '杭州市',
   '上城区劳动保障监察大队'],
  ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R2', 'R24']),
 (['浙江省',
   '杭州市',
   '上城区',
   '郭东园巷',
   '8号',
   '杭州中闽大厦',
   '北门',
   '杭州市',
   '上城区中山中路保护与综合整治指挥部'],
  ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R2', 'R24']),
 (['山东省', '泰安市', '泰山区', '花园路', '232号', '鲁郡嘉苑', '2-103号', '泰安市', '泰山区嘉洁口腔诊所'],
  ['R1', 'R2', 'R3', 'R5', 'R6', 'R30', 'R6', 'R2', 'R22']),
 (['浙江省', '温州市', '龙湾区', '永中街道', '永定路', '1188号', '万达广场', '龙湾区', '海滨街道社区学校'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R3', 'R21']),
 (['广西壮族自治区', '钦州市', '钦南区', '人民路', '翰林福第小区', '14栋', '103号', '钦州市', '钦南区春晓美容院'],
  ['R1', 'R2', 'R3', 'R5', 'R30', 'R6', 'R7', 'R2', 'R25']),
 (['新疆维吾尔自治区',
   '乌鲁木齐市',
   '水磨沟区',
   '南湖南路街道',
   '南湖南路西二巷',
   '3号',
   '新市区',
   '石油新村街道办九家湾社区居民委员会'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R3', 'R24']),
 (['浙江省',
   '杭州市',
   '滨江区',
   '滨盛路',
   '3867号',
   '滨江宝龙城市广场',
   '1期',
   '1号

In [141]:
question_data_excute[:1]

[(['江苏省', '盐城市', '亭湖区', '亭湖', '新区', '华湖路', '88号', '加州东郡', '108号楼'],
  ['R1', 'R2', 'R3', 'R3', 'R4', 'R5', 'R6', 'R30', 'R6'])]

In [186]:
excute_all_data_save = []
words_label_not_equal = []
for d in excute_all_data:
    d = d.strip()
    words, label = d.split('|')
    words = words.split(',')
    label = label.split(',')
    if len(words) == len(label):
        excute_all_data_save.append((words, label))
    else:
        words_label_not_equal.append((words, label))
        print(d)

In [190]:
excute_all_data_save[0]

(['四川省', '成都市', '龙泉驿区', '大面街道', '天鹅西湖南路', '333号', '25-2栋', '山珍菌王府'],
 ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R6', 'R30'])

In [197]:
excute_no_question_data_save = []
words_label_not_equal = []
for d in no_prop_data:
    d = d.strip()
    words, label = d.split('|')
    words = words.split(',')
    label = label.split(',')
    if len(words) == len(label):
        excute_no_question_data_save.append((words, label))
    else:
        words_label_not_equal.append((words, label))
        print(d)

In [196]:
excute_question_data_save[1]

(['香港特别行政区', '德州市', '河北邢台经济开发区', '徐梅收'], ['R1', 'R2', 'R3', 'R99'])

In [38]:
saveDataFiles_(question_data, 'data/source/r20_r22_r23_r24_R1-R4clean/question_data.pkl')

In [44]:
# saveDataFiles_(excute_all_data, 'data/source/r20_r22_r23_r24_R1-R4clean/all_76w_handle.pkl')
pkl_save_as_txt(excute_all_data, 'data/source/r20_r22_r23_r24_twoR1-R4clean/all_76w_handle.txt')

In [40]:
saveDataFiles_(no_prop_data, 'data/source/r20_r22_r23_r24_R1-R4clean/no_question_data.pkl')

In [41]:
test_all_data = readDataFiles_('data/source/0113twoR1-R4dataexcute/train_all_question_data_twoR1-R4.pkl')
len(test_all_data)

6413

In [203]:
test_question_data = readDataFiles_('data/source/1231twoR1-R4dataexcute/train_all_question_data_twoR1-R4_1231.pkl')

In [9]:
# 处理R1 R2 R3 R4 全称 与 R1 简称
len(all_data)

764336

In [104]:
all_data = readDataFiles_('data/source/1231twoR1-R4dataexcute/train_all_excute_twoR1-R4_1231.pkl')

In [219]:
from copy import deepcopy
def excute_data_fun(q_all_data, q_result):
    while True:
        items = q_all_data.get()
        if items == None:
            break
        words, labels = items
        new_labels = deepcopy(labels)
        problem = 0
        for _, w in enumerate(words):
#             if labels[_] in ['R1', 'R2', 'R3', 'R4']:
            if w in r1_r4_words and r1_r4_words_labels[w] != labels[_]:
                problem = 1
                new_labels[_] = r1_r4_words_labels[w]
        if not problem:
            q_result.put((problem, (words, labels)))
        else:
            q_result.put((problem, (words, labels, new_labels)))
    q_result.put(None)

In [220]:
q_all_data = Queue()
q_result = Queue()
# del q_all_data
# del q_result

In [221]:
process_list = []
for i in range(1):
    process_list.append(Process(target=excute_data_fun, args=(q_all_data, q_result)))

In [222]:
for temp_d in all_data:
    q_all_data.put(temp_d)
for i in range(len(process_list)):
    q_all_data.put(None)

In [223]:
excute_all_data = []
no_prop_data = []
question_data = []
question_data_excute = []

for process in process_list:
    process.start()

i = 0
end_signal = []
while not len(end_signal) == len(process_list):
    res = q_result.get()
    i+=1
    clear()
    print(i)
    if res == None:
        end_signal.append(res)
    else:
        proplem, words_labels = res
        if proplem:
            question_data.append(words_labels)
            words, labels, new_labels = words_labels
            words_labels = (words, new_labels)
        else:
            no_prop_data.append(words_labels)
        excute_all_data.append(words_labels)
#         if tag_proplem:
#             print(d)
#             question_data_excute.append(excute_d)

2


In [165]:
len(excute_all_data)

96248

In [23]:
saveDataFiles_(excute_all_data, 'data/source/0104R1-R4clean/all_clean_data.pkl')

In [8]:
all_data = readDataFiles_('data/source/0104R1-R4clean/question_data.pkl')

In [202]:
len(question_data)

146745

In [54]:
all_data = readDataFiles_('data/source/temp_0113twoR1-R4dataexcute/train_all_excute_twoR1-R4.pkl')

In [55]:
len(all_data)

764330

In [40]:
for p in process_list:
    del p

In [4]:
def splitSentence(fileAddr, encode='utf-8'):
    f=open(fileAddr,'r', encoding=encode)
    sentences_list = []
    for items in f:
        item=items.strip().replace('"','')
        line=item.split(',')
        sentences_tags=line[:len(line)]
        words = []
        tags = []
        right = True
        right_tags = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R20', 'R21', 'R22', 'R23', 'R24', 'R25', 'R30', 'R31', 'R90', 'R99']
        for i in range(len(sentences_tags)):
            if i % 2 == 0:
                words.append(sentences_tags[i])
            else:
                if sentences_tags[i] not in right_tags:
                    right = False
                    break
                tags.append(sentences_tags[i])
        if right:
            sentences_list.append((words, tags))
    return sentences_list

In [5]:
all_data = splitSentence('/home/bangsun/research/nlpAddress/data/source/dl_data_v4_21103.txt', encode='utf-8')

In [82]:
all_data[-3:]

[(['浙江', '宁波市', '余姚市', '环线', '以外', '临山镇', '城外北弄', '11号'],
  ['R1', 'R2', 'R3', 'R5', 'R90', 'R4', 'R5', 'R6']),
 (['广东', '深圳市', '龙岗区', '荷坳村', '长江埔工业区', '美平总厂'],
  ['R1', 'R2', 'R3', 'R5', 'R31', 'R20']),
 (['重庆', '沙坪坝区', '兰美路', '120号', '港城凤鸣香山', '4栋', '26-1'],
  ['R1', 'R3', 'R5', 'R6', 'R30', 'R6', 'R7'])]

In [109]:
r1_r4_full_name_question = readDataFiles_('data/source/0113twoR1-R4dataexcute/train_all_question_data_twoR1-R4.pkl')

In [110]:
r1_r4_question = readDataFiles_('data/source/temp_0113twoR1-R4dataexcute/train_all_question_data_twoR1-R4.pkl')

In [111]:
len(r1_r4_full_name_question)

6413

In [112]:
len(r1_r4_question)

106304

In [7]:
r1_r4_question[:10]

[(['内蒙古自治区', '呼和浩特市', '赛罕区', '中专路街道', '大学东街', '巨海城商住楼', '12号', '上海浦东发展银行'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R30', 'R6', 'R23']),
 (['陕西省', '西安市', '莲湖区', '环城西路街道', '劳动路', '163号', '西仪一零一小区', '2号楼'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R6']),
 (['浙江省', '杭州市', '余杭区', '南苑街道', '南兴路', '53号', '临平桂花城', '10幢'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R6']),
 (['陕西省', '西安市', '新城区', '中山门街道', '东大街', '239号', '西安国贸大厦写字楼', '陕西七方投资有限公司'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R20']),
 (['浙江省', '杭州市', '余杭区', '临平街道', '星河路', '300号', '东厦东港花苑', '23幢'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R6']),
 (['安徽省', '马鞍山市', '花山区', '湖南东路', '2633号', '朝辉东方城二区', '28栋', '1-9', '华联超市'],
  ['R1', 'R2', 'R3', 'R5', 'R6', 'R30', 'R6', 'R7', 'R25']),
 (['陕西省', '西安市', '新城区', '韩森寨街道', '万寿南路', '52号', '西安新城', '秋梅诊所'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R22']),
 (['陕西省', '西安市', '雁塔区', '等驾坡街道', '雁翔路', '99号', '华尔兹花园', '西安雁塔杜蕾口腔诊所'],
  ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R22']

In [113]:
a = r1_r4_question + r1_r4_full_name_question

In [114]:
len(a)

112717

In [115]:
def excute_repeat(data):
    for i, d in enumerate(data):
        words, tags = d
        data[i] = (tuple(words), tuple(tags))
    data = set(data)
    return data

In [120]:
r1_r4_full_name_question = excute_repeat(r1_r4_full_name_question)

In [123]:
a = set(a)

In [124]:
print(len(r1_r4_question))
print(len(r1_r4_full_name_question))
print(len(a))

102546
6298
102546


In [127]:
all_data = a - r1_r4_full_name_question

In [129]:
len(all_data)

96248

In [167]:
def list_data_to_str(words, tags):
    words_tags = []
    for _, w in enumerate(words):
        words_tags.append(w)
        words_tags.append(tags[_])
    words_tags_line = ",".join(words_tags)
    return words_tags_line

# with open('question_handle.txt', 'w')as f:
data = []
handle = []
for q_handle, q in zip(question_data_excute, question_data):
    handle_words, handle_tags = q_handle
    words, tags = q
    handle_lines = list_data_to_str(handle_words, handle_tags)
    lines = list_data_to_str(words, tags)
    data.append(handle_lines)
    handle.append(lines)
temp = pd.DataFrame({'question_data':data, 'question_handle':handle})
temp.to_excel('r1-r4two_words_question_data.xlsx')

In [132]:
all_data[:10]

[(('江苏省', '盐城市', '亭湖区', '亭湖新区', '华湖路', '88号', '加州东郡', '108号楼'),
  ('R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R6')),
 (('上海市', '普陀区', '东新路', '99弄', '新湖明珠城', '铜仁路', '90'),
  ('R1', 'R3', 'R5', 'R5', 'R30', 'R5', 'R7')),
 (('辽宁省', '抚顺市', '望花区', '营口路东段', '玫瑰城', '29号', '4-5', '辽宁沈抚农村商业银行'),
  ('R1', 'R2', 'R3', 'R5', 'R31', 'R6', 'R7', 'R23')),
 (('山东省',
   '东营市',
   '垦利区',
   '明苑南路',
   '通明苑小区',
   '胜利石油管理局胜北社区',
   '电力幼儿园',
   '通明苑小区',
   '25号楼'),
  ('R1', 'R2', 'R3', 'R5', 'R30', 'R5', 'R21', 'R30', 'R6')),
 (('上海市',
   '闵行区',
   '顾戴路',
   '1325弄',
   '70号',
   '101号室',
   '江南星城',
   '二期',
   '谷丽阁美甲美睫定制SPA会所'),
  ('R1', 'R3', 'R5', 'R5', 'R6', 'R7', 'R30', 'R6', 'R25')),
 (('四川省', '绵阳市', '三台县', '塔山镇', '雄关干道', '100-1号', '三台县公安局交通警察大队五中队'),
  ('R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R24')),
 (('浙江省', '台州市', '黄岩区', '滨江新村', '69幢', '3单元', '402室', '台州市黄岩浩远医用塑料有限公司'),
  ('R1', 'R2', 'R3', 'R30', 'R6', 'R7', 'R7', 'R20')),
 (('广东省', '广州市', '越秀区', '环市中路', '321号', '广东三建综合办公大楼', '3层', '利荣五金装饰材料店'),


In [42]:
# 清洗pkl数据

In [49]:
def pkl_save_as_txt(pkl_data, save_path):
    with open(save_path, 'w')as f:
        for data in pkl_data:
            words, tags = data
            if len(words) != len(tags):
                print(data)
                continue
            sen = []
            for i, word in enumerate(words):
                sen.append(words[i])
                sen.append(tags[i])
            sen_str = ','.join(sen)
            f.write(sen_str + '\n')
            
def read_rectify_file(path):
    rectify_data = pd.read_excel(path, sheet_name='修正')
    error_data = pd.read_excel(path, sheet_name='删除')
    return rectify_data, error_data

def read_76w_data(_76w_path):
    with open(_76w_path, 'r')as f:
        temp = f.readlines()

    all_data = []
    for line in temp:
        line = line.strip()
        line = line.split(',')
        words = [l for _, l in enumerate(line) if _%2==0]
        tags = [l for _, l in enumerate(line) if _%2==1]
        if len(words) != len(tags):
            print(line)
            continue
        all_data.append((words, tags))
    return all_data

def create_pkl_words_map(pkl_tag, all_data):
    word_data_map = {}
    for _, data in enumerate(all_data):
        words, tags = data
        if pkl_tag in tags:
            tag_index = []
            for index, t in enumerate(tags):
                if t==pkl_tag:
                    tag_index.append(index)
                    
            for index in tag_index:
                word = words[index]
                if word in word_data_map:
                    word_data_map[word].append(_)
                else:
                    word_data_map[word] = [_]
    return word_data_map

def handle_rectify_data(row):
    error = 0
    replace_words = []
    replace_tags = []
    rectify = row['rectify']
    rectify = str(rectify)
    rectify = rectify.strip()
    if rectify.isdigit():
        replace_words.append(row['words'])
        replace_tags.append('R'+rectify)
    else:
        rectify = [i for i in rectify.split(';') if i]
        for r in rectify:
            r = r.strip()
            r = r.split(',')
            if len(r) == 2 and r[1].isdigit():
                replace_words.append(r[0])
                replace_tags.append('R'+r[1])
            else:
                error = r
                break
    return replace_words, replace_tags, error

def rectify_76w_data(word_data_map, _76w_all_data, rectify_data):
    re_num = 0
    is_rectify_list = []
    errors = []
    for _, row in rectify_data.iterrows():
        re_word = row['words']
        is_rectify = -1
        if re_word in word_data_map:
            w_index = word_data_map[re_word]
            is_rectify = w_index
            replace_words, replace_tags, error = handle_rectify_data(row)
            if error:
                errors.append(error)
                continue
                
            for index in w_index:
                new_words = []
                new_tags = []
                target_words, target_tags = _76w_all_data[index]
                for w_index, w in enumerate(target_words):
                    if w==re_word:
                        new_words.extend(replace_words)
                        new_tags.extend(replace_tags)
                    else:
                        new_words.append(w)
                        new_tags.append(target_tags[w_index])
                if not len(new_words) == len(new_tags):
                    print('*'*100)
                    print((new_words, new_tags))
                    print()
                    raise ValueError()
                _76w_all_data[index] = (new_words, new_tags)
                re_num += 1
                print(re_num)
                print((target_words, target_tags))
                print(_76w_all_data[index])
                print()
                
        is_rectify_list.append(is_rectify)
    print('*'*50)
    print(errors)
    return _76w_all_data, is_rectify_list

def del_76w_error_data(word_data_map, _76w_all_data, rectify_data):
    del_num = 0
    is_del_list = []
    for _, row in rectify_data.iterrows():
        is_del = -1
        del_word = row['words']
        if del_word in word_data_map:
            w_index = word_data_map[del_word]
            is_del = w_index
            for index in w_index:
                new_words = []
                new_tags = []
                target_words, target_tags = _76w_all_data[index]
                for w_index, w in enumerate(target_words):
                    if not w==del_word:
                        new_words.append(w)
                        new_tags.append(target_tags[w_index])
                if not len(new_words) == len(new_tags):
                    print('*'*100)
                    print((new_words, new_tags))
                    print()
                    raise ValueError()
                _76w_all_data[index] = (new_words, new_tags)
                del_num += 1
                print(del_num)
                print((target_words, target_tags))
                print(_76w_all_data[index])
                print()
        is_del_list.append(is_del)
    return _76w_all_data, is_del_list

In [3]:
_76w_all_data= readDataFiles_('data/source/0104R1-R4clean/all_clean_data.pkl')
_76w_all_data_copy = copy.deepcopy(_76w_all_data)

In [4]:
# 清洗r22

r22_rectify_data, r22_error_data = read_rectify_file('data/source/rectify_pkl/r22/r22.xlsx')
r22_word_data_map = create_pkl_words_map('R22', _76w_all_data_copy)

In [5]:
r22_new_76w_data, r22_is_rectify_list = rectify_76w_data(r22_word_data_map, _76w_all_data, r22_rectify_data)
r22_rectify_data['is_rectify'] = r22_is_rectify_list
r22_rectify_data.to_excel('data/source/rectify_pkl/r22/r22_rectify.xlsx')

1
(['山东省', '青岛市', '黄岛区', '五台山路', '1677号', '青岛大学医学院附属医院', '黄岛分院', 'A楼', 'F1', '青岛大学医学院附属医院综合服务大厅'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R22', 'R22', 'R7', 'R7', 'R22'])
(['山东省', '青岛市', '黄岛区', '五台山路', '1677号', '青岛大学医学院附属医院', '黄岛分院', 'A楼', 'F1', '青岛大学医学院附属医院', '综合服务大厅'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R22', 'R22', 'R7', 'R7', 'R22', 'R6'])

2
(['河南省', '郑州市', '二七区', '永安街', '35号', '永安小区', '5号楼', '郑州大学第五附属医院住院医师规范化培训基层实践基地'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R30', 'R6', 'R22'])
(['河南省', '郑州市', '二七区', '永安街', '35号', '永安小区', '5号楼', '郑州大学第五附属医院住院医师规范化培训基层实践基地'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R30', 'R6', 'R31'])

3
(['山东省', '济南市', '历下区', '舜华路', '2000号', '舜泰广场', '2号楼', '8层', '801', '中国重型汽车集团有限公司医院车桥医务室'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R6', 'R7', 'R7', 'R22'])
(['山东省', '济南市', '历下区', '舜华路', '2000号', '舜泰广场', '2号楼', '8层', '801', '中国重型汽车集团有限公司医院', '车桥医务室'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R6', 'R7', 'R7', 'R22', 'R22'])

4
(['山东省', '临沂市', '沂水县', '沂城街道', '健康东路', '第三中学沂城卫生院后埠东卫生室', '二楼', '泰

In [6]:
r22_del_76w_data, r22_is_del_list = del_76w_error_data(r22_word_data_map, r22_new_76w_data, r22_error_data)
r22_error_data['is_del'] = r22_is_del_list
r22_error_data.to_excel('data/source/rectify_pkl/r22/r22_del.xlsx')

1
(['上海市', '闵行区', '吴中路', '1100号', '炫润国际大厦', '6层', '上海中医药大学固生堂万嘉门诊部'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R7', 'R22'])
(['上海市', '闵行区', '吴中路', '1100号', '炫润国际大厦', '6层'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R7'])



In [7]:
# 清洗r23

r23_rectify_data, r23_error_data = read_rectify_file('data/source/rectify_pkl/r23/r23.xlsx')
r23_word_data_map = create_pkl_words_map('R23', _76w_all_data_copy)

In [8]:
r23_new_76w_data, r23_is_rectify_list = rectify_76w_data(r23_word_data_map, r22_del_76w_data, r23_rectify_data)
r23_rectify_data['is_rectify'] = r23_is_rectify_list
r23_rectify_data.to_excel('data/source/rectify_pkl/r23/r23_rectify.xlsx')

1
(['云南省', '昆明市', '安宁市', '连然街道', '云化社区', '康云路', '8号', '安宁市人民医院', '停车场'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R23', 'R25'])
(['云南省', '昆明市', '安宁市', '连然街道', '云化社区', '康云路', '8号', '安宁市人民医院', '停车场'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R22', 'R25'])

2
(['山东省', '青岛市', '市南区', '八大关街道', '武昌路', '1号乙', '3号', '八大关街道办事处', '八大关社区文化活动中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R6', 'R23', 'R24'])
(['山东省', '青岛市', '市南区', '八大关街道', '武昌路', '1号乙', '3号', '八大关街道办事处', '八大关社区文化活动中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R6', 'R24', 'R24'])

3
(['四川省', '成都市', '成华区', '荆顺东街', '259号', '2栋', '百信集团杏仁堂大药房'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R6', 'R23'])
(['四川省', '成都市', '成华区', '荆顺东街', '259号', '2栋', '百信集团杏仁堂大药房'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R6', 'R25'])

4
(['四川省', '成都市', '成华区', '荆顺东街', '259号', '附9号', '百信药业成华区正方大药房'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R6', 'R23'])
(['四川省', '成都市', '成华区', '荆顺东街', '259号', '附9号', '百信药业成华区正方大药房'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R6', 'R25'])

5
(['广东省', '佛山市', '南海区', '狮山

(['山西省', '运城市', '盐湖区', '人民北路', '5号', '锦绣花城', '北区', '6号', '山西省', '绛县农村商业银行'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R30', 'R6', 'R7', 'R1', 'R23'])

336
(['山西省', '临汾市', '尧都区', '南外环路', '神州装饰城', '北门', '17号', '山西省尧都区农村商业银行公司县底支行'], ['R1', 'R2', 'R3', 'R5', 'R31', 'R7', 'R7', 'R23'])
(['山西省', '临汾市', '尧都区', '南外环路', '神州装饰城', '北门', '17号', '山西省', '尧都区', '农村商业银行公司县底支行'], ['R1', 'R2', 'R3', 'R5', 'R31', 'R7', 'R7', 'R1', 'R3', 'R23'])

337
(['山西省', '太原市', '小店区', '平阳路', '亲贤街', '口西南角平阳景苑', '20栋', '山西省尧都区农村商业银行亲贤街小微企业专营支行'], ['R1', 'R2', 'R3', 'R5', 'R5', 'R30', 'R6', 'R23'])
(['山西省', '太原市', '小店区', '平阳路', '亲贤街', '口西南角平阳景苑', '20栋', '山西省', '尧都区', '农村商业银行亲贤街小微企业专营支行'], ['R1', 'R2', 'R3', 'R5', 'R5', 'R30', 'R6', 'R1', 'R3', 'R23'])

338
(['安徽省', '合肥市', '庐阳区', '大杨镇', '四里河路', '88号', '明发商业广场', 'C3', '商中国工商银行'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R7', 'R23'])
(['安徽省', '合肥市', '庐阳区', '大杨镇', '四里河路', '88号', '明发商业广场', 'C3', '中国工商银行'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R7', 'R23'])

339
(['河北省', '

(['重庆市', '九龙坡区', '杨家坪街道', '杨家坪正街', '20号', '重百商场', '中国建设银行'], ['R1', 'R3', 'R4', 'R5', 'R6', 'R31', 'R23'])

651
(['重庆市', '渝北区', '金开大道', '1003号', '重庆爱琴海购物公园万瑞大平层售房部农业银行', '招商银行24小时自助银行两江支行'], ['R1', 'R3', 'R5', 'R6', 'R23', 'R23'])
(['重庆市', '渝北区', '金开大道', '1003号', '重庆爱琴海购物公园', '万瑞大平层售房部', '农业银行', '招商银行24小时自助银行两江支行'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R25', 'R23', 'R23'])

652
(['重庆市', '渝北区', '金开大道', '1003号', '重庆爱琴海购物公园万瑞大平层售房部农业银行', '华夏银行ATM'], ['R1', 'R3', 'R5', 'R6', 'R23', 'R25'])
(['重庆市', '渝北区', '金开大道', '1003号', '重庆爱琴海购物公园', '万瑞大平层售房部', '农业银行', '华夏银行ATM'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R25', 'R23', 'R25'])

653
(['江苏省', '徐州市', '泉山区', '永安街道', '西安南路', '欧洲广场', 'C', '撞淮海银行'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R31', 'R7', 'R23'])
(['江苏省', '徐州市', '泉山区', '永安街道', '西安南路', '欧洲广场', 'C', '淮海银行'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R31', 'R7', 'R23'])

654
(['广东省', '深圳市', '福田区', '福中路', '42号', '紫玉花园招商银行', '二楼', '健康家养生会所'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R23', 'R7', 'R25'])
(['广东省', '深圳市', '福田区', '福

In [9]:
r23_del_76w_data, r23_is_del_list = del_76w_error_data(r23_word_data_map, r23_new_76w_data, r23_error_data)
r23_error_data['is_del'] = r23_is_del_list
r23_error_data.to_excel('data/source/rectify_pkl/r23/r23_del.xlsx')

1
(['河南省', '南阳市', '卧龙区', '光武街道', '光武中路', '1355号', '毛主席像', '河南省农村信用社光武分社'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R23', 'R23'])
(['河南省', '南阳市', '卧龙区', '光武街道', '光武中路', '1355号', '河南省农村信用社光武分社'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R23'])

2
(['江苏省', '南通市', '崇川区', '城东街道', '人民东路', '8号', '王府大厦', '麒玉楼交银康联'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R23'])
(['江苏省', '南通市', '崇川区', '城东街道', '人民东路', '8号', '王府大厦'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31'])

3
(['四川省', '资阳市', '雁江区', '石岭镇', '金带铺', '中国工商银行ATM', '西门', '桥储蓄所'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R25', 'R7', 'R23'])
(['四川省', '资阳市', '雁江区', '石岭镇', '金带铺', '中国工商银行ATM', '西门'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R25', 'R7'])

4
(['新疆维吾尔自治区', '喀什地区', '莎车县', '莎车镇', '新城路', '12号', '麦盖提县', '45', '团农村信用社'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R3', 'R7', 'R23'])
(['新疆维吾尔自治区', '喀什地区', '莎车县', '莎车镇', '新城路', '12号', '麦盖提县', '45'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R3', 'R7'])

5
(['辽宁省', '丹东市', '振兴区', '桃源街', '14号', '中国工商银行宽甸', '南门', '外储蓄所'], ['R1'

In [10]:
# 清洗R20

r20_rectify_data, r20_error_data = read_rectify_file('data/source/rectify_pkl/r20/r20.xlsx')
r20_word_data_map = create_pkl_words_map('R20', _76w_all_data_copy)

In [11]:
r20_new_76w_data, r20_is_rectify_list = rectify_76w_data(r20_word_data_map, r23_del_76w_data, r20_rectify_data)
r20_rectify_data['is_rectify'] = r20_is_rectify_list
r20_rectify_data.to_excel('data/source/rectify_pkl/r20/r20_rectify.xlsx')

1
(['广东省', '惠州市', '惠东县', '多祝镇', '新建路', '38号', '多祝镇大道村委会', '计划生育协会'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R24', 'R20'])
(['广东省', '惠州市', '惠东县', '多祝镇', '新建路', '38号', '多祝镇大道村委会', '计划生育协会'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R24', 'R24'])

2
(['四川省', '自贡市', '富顺县', '宝庆乡', '计划生育协会'], ['R1', 'R2', 'R3', 'R4', 'R20'])
(['四川省', '自贡市', '富顺县', '宝庆乡', '计划生育协会'], ['R1', 'R2', 'R3', 'R4', 'R24'])

3
(['广东省', '湛江市', '雷州市', '客路镇', '中华街', '26号', '客路镇迈哉村委会', '计划生育协会'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R24', 'R20'])
(['广东省', '湛江市', '雷州市', '客路镇', '中华街', '26号', '客路镇迈哉村委会', '计划生育协会'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R24', 'R24'])

4
(['重庆市', '北碚区', '天府镇', '天府车站路', '五新村', '计划生育协会'], ['R1', 'R3', 'R4', 'R5', 'R5', 'R20'])
(['重庆市', '北碚区', '天府镇', '天府车站路', '五新村', '计划生育协会'], ['R1', 'R3', 'R4', 'R5', 'R5', 'R24'])

5
(['四川省', '成都市', '邛崃市', '高埂镇', '新桥街', '高埂村', '计划生育协会'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R20'])
(['四川省', '成都市', '邛崃市', '高埂镇', '新桥街', '高埂村', '计划生育协会'], ['R1', 'R2', 'R3', 'R4', 'R5'

296
(['江苏省', '苏州市', '吴江区', '震泽镇', '镇南一路', '1381号', '阳光家园', '震泽行政服务中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R20'])
(['江苏省', '苏州市', '吴江区', '震泽镇', '镇南一路', '1381号', '阳光家园', '震泽行政服务中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R24'])

297
(['广东省', '佛山市', '禅城区', '南庄镇', '季华西路', '131号', '绿岛湖行政服务中心', '1号楼'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R20', 'R6'])
(['广东省', '佛山市', '禅城区', '南庄镇', '季华西路', '131号', '绿岛湖行政服务中心', '1号楼'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R24', 'R6'])

298
(['江苏省', '苏州市', '昆山市', '锦溪镇', '锦溪行政服务中心', '2栋', '2F', '昆山市锦溪镇劳动和社会保障所'], ['R1', 'R2', 'R3', 'R4', 'R20', 'R6', 'R7', 'R24'])
(['江苏省', '苏州市', '昆山市', '锦溪镇', '锦溪行政服务中心', '2栋', '2F', '昆山市锦溪镇劳动和社会保障所'], ['R1', 'R2', 'R3', 'R4', 'R24', 'R6', 'R7', 'R24'])

299
(['浙江省', '金华市', '东阳市', '吴宁街道', '东街', '吴宁东路', '217号', '东城街道行政服务中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R20'])
(['浙江省', '金华市', '东阳市', '吴宁街道', '东街', '吴宁东路', '217号', '东城街道行政服务中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R24'])

300
(['浙江省', '杭州

707
(['江苏省', '南通市', '崇川区', '和平桥街道', '人民中路', '28号', '南通大厦', 'D座', '618室', '金球英语教育中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R6', 'R7', 'R20'])
(['江苏省', '南通市', '崇川区', '和平桥街道', '人民中路', '28号', '南通大厦', 'D座', '618室', '金球英语教育中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R6', 'R7', 'R21'])

708
(['天津市', '河西区', '南京路', '66号', '凯旋门百货', 'B座', '30A室', '乐知舟儿童英语教育中心'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R6', 'R7', 'R20'])
(['天津市', '河西区', '南京路', '66号', '凯旋门百货', 'B座', '30A室', '乐知舟儿童英语教育中心'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R6', 'R7', 'R21'])

709
(['广东省', '深圳市', '南山区', '留仙大道', '2188号', '天悦湾花园悦方广场', '2层', 'L241号', '英火虫英语教育中心'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R7', 'R20'])
(['广东省', '深圳市', '南山区', '留仙大道', '2188号', '天悦湾花园悦方广场', '2层', 'L241号', '英火虫英语教育中心'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R7', 'R21'])

710
(['山西省', '太原市', '迎泽区', '柳巷街道', '开化寺街', '87号', '华宇商贸写字楼', '英豪英语教育中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R20'])
(['山西省', '太原市', '迎泽区', '柳巷街道', '开化寺街', '87号',

1006
(['北京市', '朝阳区', '东三环中路', '16号', '京粮大厦', '1202室', '中宠宠物及用品发展服务中心'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R7', 'R20'])
(['北京市', '朝阳区', '东三环中路', '16号', '京粮大厦', '1202室', '中宠宠物及用品发展服务中心'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R7', 'R24'])

1007
(['湖南省', '张家界市', '永定区', '古庸路', '572号', '金领国际大厦', '17层', '张家界世纪青少年发展服务中心'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R20'])
(['湖南省', '张家界市', '永定区', '古庸路', '572号', '金领国际大厦', '17层', '张家界世纪青少年发展服务中心'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R24'])

1008
(['湖南省', '长沙市', '芙蓉区', '东二环一段', '1079号', '锦泰商务大厦', '3层', '304-305', '长沙社保公积金代理华山服务中心'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R7', 'R20'])
(['湖南省', '长沙市', '芙蓉区', '东二环一段', '1079号', '锦泰商务大厦', '3层', '304-305', '长沙社保公积金代理华山服务中心'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R7', 'R24'])

1009
(['广东省', '江门市', '鹤山市', '沙坪街道', '新城路', '300号', '幸福家园', '社工服务中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R20'])
(['广东省', '江门市', '鹤山市', '沙坪街道', '新城路', '300号', '幸福家园', '社工服务中心'], ['R1', 'R2', 'R3', 'R4'

(['四川省', '成都市', '温江区', '新南路', '268号', '成都本草堂药品连锁有限公司温江青平大药房'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R20'])
(['四川省', '成都市', '温江区', '新南路', '268号', '成都本草堂药品连锁有限公司温江青平大药房'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R25'])

1535
(['云南省', '普洱市', '宁洱哈尼族彝族自治县', '宁洱镇', '宏远路', '8号', '永康药业宁洱永康药业有限公司健康大药房'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R20'])
(['云南省', '普洱市', '宁洱哈尼族彝族自治县', '宁洱镇', '宏远路', '8号', '永康药业宁洱永康药业有限公司健康大药房'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R25'])

1536
(['四川省', '成都市', '成华区', '龙潭街道', '华冠路', '4号', '附32号', '贝尔康大药房连锁有限公司爱心大药房'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R6', 'R20'])
(['四川省', '成都市', '成华区', '龙潭街道', '华冠路', '4号', '附32号', '贝尔康大药房连锁有限公司爱心大药房'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R6', 'R25'])

1537
(['四川省', '成都市', '双流区', '乌江路', '352号', '四川康贝大药房连锁有限公司龙港大药房'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R20'])
(['四川省', '成都市', '双流区', '乌江路', '352号', '四川康贝大药房连锁有限公司龙港大药房'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R25'])

1538
(['四川省', '成都市', '双流区', '菜蔬街二段', '69号', '成都本草堂药品连锁有限公司仁福大药房'], ['R1', 'R2', 'R3', 'R5

1828
(['山东省', '东营市', '东营区', '文汇街道', '济南路', '35号', '移动公司旁邮电小区', '黄二棍菜馆'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R20'])
(['山东省', '东营市', '东营区', '文汇街道', '济南路', '35号', '移动公司旁邮电小区', '黄二棍菜馆'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R30', 'R25'])

1829
(['辽宁省', '沈阳市', '沈河区', '中街路', '205号', 'HK01号', '流行馆', 'F6', 'HOTKEKE专柜'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R6', 'R20', 'R7', 'R25'])
(['辽宁省', '沈阳市', '沈河区', '中街路', '205号', 'HK01号', '流行馆', 'F6', 'HOTKEKE专柜'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R6', 'R25', 'R7', 'R25'])

1830
(['辽宁省', '沈阳市', '沈河区', '中街路', '205号', 'HK01号', '流行馆', 'F7', 'GENANX专柜'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R6', 'R20', 'R7', 'R25'])
(['辽宁省', '沈阳市', '沈河区', '中街路', '205号', 'HK01号', '流行馆', 'F7', 'GENANX专柜'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R6', 'R25', 'R7', 'R25'])

1831
(['山东省', '青岛市', '李沧区', '李村街道', '书院路', '186号', '达翁家居软装馆', '一楼', 'A08', '青岛信合盘石网络信息技术有限公司'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R20', 'R7', 'R7', 'R20'])
(['山东省', '青岛市', '李沧区', '李村街道', '书院路', '186号', '达翁家居软装馆',

2091
(['山东省', '潍坊市', '高密市', '密水街道', '杏坛西街', '湖滨花园', '39号楼', '租售咨询处'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R30', 'R6', 'R25'])
(['山东省', '潍坊市', '高密市', '密水街道', '杏坛西街', '湖滨花园', '39号楼', '租售咨询处'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R30', 'R6', 'R25'])

2092
(['上海市', '浦东新区', '年家浜路', '518号', '万达广场', 'B1层', '一兆韦德咨询处'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R7', 'R20'])
(['上海市', '浦东新区', '年家浜路', '518号', '万达广场', 'B1层', '一兆韦德咨询处'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R7', 'R25'])

2093
(['四川省', '攀枝花市', '仁和区', '土城街', '21号', '四川法德美会计师事务所咨询处'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R20'])
(['四川省', '攀枝花市', '仁和区', '土城街', '21号', '四川法德美会计师事务所', '咨询处'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R20', 'R6'])

2094
(['天津市', '河东区', '津滨大道', '160号', '天津爱琴海购物公园', 'F5层', '世纪星滑冰学校咨询处'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R7', 'R20'])
(['天津市', '河东区', '津滨大道', '160号', '天津爱琴海购物公园', 'F5层', '世纪星滑冰学校'], ['R1', 'R3', 'R5', 'R6', 'R31', 'R7', 'R20'])

2095
(['安徽省', '合肥市', '瑶海区', '经五路', '勤居苑', '东门', '103门', '五星驾校咨询处'], ['R1', 'R2', 'R3', 'R5', 'R30', 'R7',


2387
(['江苏省', '无锡市', '宜兴市', '新街街道', '绿园路', '489号', '1幢', '华仁建设集团有限公司化工成套设备厂-工程技术中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R6', 'R20'])
(['江苏省', '无锡市', '宜兴市', '新街街道', '绿园路', '489号', '1幢', '华仁建设集团有限公司化工成套设备厂工程技术中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R6', 'R20'])

2388
(['四川省', '成都市', '金牛区', '一环路南一段', '24号', '四川大学望江校区', '中国测试技术研究院-水污染诊断及控制工程技术中心'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R21', 'R20'])
(['四川省', '成都市', '金牛区', '一环路南一段', '24号', '四川大学望江校区', '中国测试技术研究院', '-水污染诊断及控制工程技术中心'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R21', 'R20', 'R20'])

2389
(['山东省', '青岛市', '崂山区', '中韩街道', '宁夏路', '308号', '青岛大学', '青岛大学现代教育技术中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R21', 'R20'])
(['山东省', '青岛市', '崂山区', '中韩街道', '宁夏路', '308号', '青岛大学', '青岛大学现代教育技术中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R21', 'R21'])

2390
(['云南省', '昆明市', '五华区', '华山街道', '青云街', '50号', '昆明糖果糕点大楼', '昆明理工大学现代教育技术中心'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R20'])
(['云南省', '昆明市', '五华区', '华山街道', '青云街', '50号', '昆明糖果糕点大楼', '昆明理工大学现代教育技术中心']

2774
(['广东省', '广州市', '天河区', '华南理工大学国家大学科技园金华园区', '7层', 'C704', 'C706', '华南师大网络教育学院'], ['R1', 'R2', 'R3', 'R31', 'R7', 'R7', 'R7', 'R20'])
(['广东省', '广州市', '天河区', '华南理工大学国家大学科技园金华园区', '7层', 'C704', 'C706', '华南师大网络教育学院'], ['R1', 'R2', 'R3', 'R31', 'R7', 'R7', 'R7', 'R21'])

2775
(['山东省', '潍坊市', '奎文区', '鸢飞路', '626号', '中友大厦', '806室', '北京语言大学网络教育学院'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R20'])
(['山东省', '潍坊市', '奎文区', '鸢飞路', '626号', '中友大厦', '806室', '北京语言大学网络教育学院'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R21'])

2776
(['广东省', '广州市', '天河区', '五山路', '178号', '华南师范大学科技大楼', '1层', '128-129室', '广东商学院网络教育学院'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R7', 'R20'])
(['广东省', '广州市', '天河区', '五山路', '178号', '华南师范大学科技大楼', '1层', '128-129室', '广东商学院网络教育学院'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R31', 'R7', 'R7', 'R21'])

2777
(['山东省', '济南市', '历城区', '山大路街道', '山大南路', '29-1号', '山大鲁能科技大厦', '山东大学-继续教育学院'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R20'])
(['山东省', '济南市', '历城区', '山大路街道', '山大南路', '29-1号',

In [12]:
r20_del_76w_data, r20_is_del_list = del_76w_error_data(r20_word_data_map, r20_new_76w_data, r20_error_data)
r20_error_data['is_del'] = r20_is_del_list
r20_error_data.to_excel('data/source/rectify_pkl/r20/r20_del.xlsx')

1
(['江苏省', '南京市', '建邺区', '南湖路', '40号', '际华三五0', '南京金现汽车销售有限公司', '停车场'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R20', 'R20', 'R25'])
(['江苏省', '南京市', '建邺区', '南湖路', '40号', '南京金现汽车销售有限公司', '停车场'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R20', 'R25'])

2
(['湖北省', '武汉市', '洪山区', '海淀街道', '万泉河路', '三义庙2号院', '达内外企IT'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R30', 'R20'])
(['湖北省', '武汉市', '洪山区', '海淀街道', '万泉河路', '三义庙2号院'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R30'])

3
(['广东省', '广州市', '越秀区', '流花街道', '环市中路', '207号', '恒生大厦', 'FavorInterlinkLtd'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R20'])
(['广东省', '广州市', '越秀区', '流花街道', '环市中路', '207号', '恒生大厦'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31'])

4
(['广东省', '广州市', '越秀区', '洪桥街道', '小北路', '187号', '鹏源发展大厦', 'ValenciaTradingCoLtd'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31', 'R20'])
(['广东省', '广州市', '越秀区', '洪桥街道', '小北路', '187号', '鹏源发展大厦'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R31'])

5
(['广东省', '汕头市', '金平区', '东方街道', '金砂路', '86号', '友谊国际大厦', 'ShantouWeiboFootwearCoLtd'], ['R1', 'R2'

In [13]:
# 清洗r24

r24_rectify_data, r24_error_data = read_rectify_file('data/source/rectify_pkl/r24/r24.xlsx')
r24_word_data_map = create_pkl_words_map('R24', _76w_all_data_copy)

In [14]:
r24_new_76w_data, r24_is_rectify_list = rectify_76w_data(r24_word_data_map, r20_del_76w_data, r24_rectify_data)
r24_rectify_data['is_rectify'] = r24_is_rectify_list
r24_rectify_data.to_excel('data/source/rectify_pkl/r24/r24_rectify.xlsx')

**************************************************
[]


In [15]:
r24_del_76w_data, r24_is_del_list = del_76w_error_data(r24_word_data_map, r24_new_76w_data, r24_error_data)
r24_error_data['is_del'] = r24_is_del_list
r24_error_data.to_excel('data/source/rectify_pkl/r24/r24_del.xlsx')

In [19]:
saveDataFiles_(r24_del_76w_data, 'data/source/76wdata_r20_r22_r23_r24clear/train_all_data.pkl')
pkl_save_as_txt(r24_del_76w_data, 'data/source/76wdata_r20_r22_r23_r24clear/76wdata.txt')

In [50]:
_76w = read_76w_data('data/source/r20_r22_r23_r24_twoR1-R4clean/all_76w_handle.txt')
saveDataFiles_(_76w, 'data/source/r20_r22_r23_r24_twoR1-R4clean/train_all_data.pkl')

In [75]:
def pkl_to_list(word_data_map, tag):
    words = list(word_data_map.keys())
    words_list = []
    for w in words:
        words_list.append([[w], [tag]])
    return words_list

In [44]:
# R22级别提取
# 词内存在街道、路或者长度大于12或者出现底商、沿街、医院医院就认为有问题
def get_r22_right_wrong_data(r22):
    rightList = []
    wrongList = []
    ruleList = [
        "'街道' in word",
        "'路' in word",
        "'医院医院' in word",
        "'底商' in word",
        "'沿街' in word",
        "'西头' in word",
        "'商业用房' in word",
        "len(word) > 12",
        "'校本部' in word"
    ]
    for item in r22:
        word = item[0][0]
        wrong = False
        for rule in ruleList:
            if (eval(rule)):
                wrong = True
                break
        if wrong:
            wrongList.append(word)
        else:
            rightList.append(word)
    rightList, wrongList = list(set(rightList)), list(set(wrongList))
    print(len(rightList), len(wrongList))
    return rightList, wrongList

In [117]:
rl = readDataFiles_('/home/bangsun/research/nlpAddress/data/generate/pkl/new/source/r22_hospital_1110.pkl')
r22_word_data_map = create_pkl_words_map('R22', _76w_all_data)
r22_list = pkl_to_list(r22_word_data_map, 'R22')
old_right, old_wrong = get_r22_right_wrong_data(rl)
new_right, new_wrong = get_r22_right_wrong_data(r22_list)
old_wrong = set(old_wrong)
new_wrong = set(new_wrong)
temp = new_wrong - old_wrong
print(len(temp))

5322 398
8220 1553
1155


In [90]:
temp

{'北京市昌平区中西医结合医院',
 '柯桥区齐贤镇新街社区卫生服务站',
 '华北油田物探社区卫生服务站',
 '南坪街道阳光社区卫生服务站',
 '深圳市龙华区观澜预防保健所',
 '黄巷街道社区卫生服务中心',
 '巴南区供销合作社联合社庄稼医院',
 '隆庆路社区卫生服务站',
 '荣昌区仁义镇鹿子村红十字会',
 '辽宁省人民医院-计算机信息中心',
 '花园路街道牤牛桥社区医疗卫生站',
 '太原市民营经济开发区于兆虎口腔诊所',
 '东南大学附属中大医院江北院区',
 '富春街道秋月社区卫生服务站',
 '天竺镇卫生院二十里堡社区卫生服务站',
 '向阳街道办事处所城社区卫生站',
 '城乡居民基本医疗保险定点医院',
 '新会区双水镇中心卫生院红十字会',
 '新平彝族傣族自治县人民医院',
 '重庆市巴南区一品街道社区卫生服务中心',
 '虞山镇莫城社区卫生服务中心',
 '上海市黄浦区五里桥街道社区卫生服务中心江南社区卫生服务站',
 '沙坪坝区童家桥社区卫生服务中心',
 '来广营社区卫生服务中心-预防接种门诊',
 '南川区城区动物卫生监督分所',
 '青州市民政局卫生所-门诊部',
 '静海县动物卫生监督所双塘分所',
 '张家窝镇社区卫生服务中心预防接种门诊',
 '重庆市涪陵区石沱镇中心卫生院',
 '广瑞通江街道社区卫生服务中心',
 '攀枝花市铁路医院',
 '重庆市万州区人民医院龙宝分院',
 '淄博高新市八院新空间门诊部',
 '花园路社区卫生服务站',
 '东城区龙潭街道板厂南里社区红十字会服务站',
 '张店区马尚镇紫园社区卫生服务站',
 '潍城区南关街道计划生育服务站',
 '秀山土家族苗族自治县人民医院门诊部',
 '大渡口区八桥镇五一村卫生室',
 '北辛街道新华街社区卫生服务站',
 '江津区朱杨镇卫生院公共卫生服务中心',
 '青阳镇芦塘村社区卫生服务站',
 '清河办事处社区卫生服务中心',
 '东城街道社区卫生服务中心门诊部',
 '重庆市北碚区静观镇中心卫生院',
 '成都市妇女儿童中心医院太升南路院区',
 '黄阁镇社区卫生服务中心预防接种门诊',
 '江北区复盛镇计划生育服务站',
 '解放路街社区卫生服务中心',
 '沙坪坝区石井坡社区卫生服务中心',
 '深圳市罗湖区翠竹所人口和计划生育服务站',


In [78]:
# R20级别提取
# 后缀是公司、集团、厂、事务所、办事处、农场、企业（长度小于7）、商会（长度小于6）、电视台（长度小于6）的全部提
def get_r20_right_wrong_data(r20):
    rightList = []
    wrongList = []
    ruleList = [
        "re.match('.*公司$', word) != None",
        "re.match('.*厂$', word) != None",
        "re.match('.*事务所$', word) != None",
        "re.match('.*集团$', word) != None",
        "re.match('.*办事处$', word) != None",
        "re.match('.*农场$', word) != None",
        "re.match('.*企业$', word) != None and len(word) <= 6",
        "re.match('.*商会$', word) != None and len(word) <= 5",
        "re.match('.*电视台$', word) != None and len(word) <= 5"
    ]
    for item in r20:
        word = item[0][0]
        right = False
        for rule in ruleList:
            if(eval(rule)):
                right = True
                break
        if right:
            rightList.append(word)
        else:
            wrongList.append(word)
    rightList, wrongList = list(set(rightList)), list(set(wrongList))
    print(len(rightList), len(wrongList))
    return rightList, wrongList

In [86]:
rl = readDataFiles_('/home/bangsun/research/nlpAddress/data/generate/pkl/new/source/r20_company_1110.pkl')
r20_word_data_map = create_pkl_words_map('R20', _76w_all_data)
r20_list = pkl_to_list(r20_word_data_map, 'R20')
old_right, old_wrong = get_r20_right_wrong_data(rl)
new_right, new_wrong = get_r20_right_wrong_data(r20_list)
old_wrong = set(old_wrong)
new_wrong = set(new_wrong)
temp = new_wrong - old_wrong
print(len(temp))

142608 11903
169811 16916
5013


In [134]:
# R23级别提取
rightList = ['邯郸银行', '中国人民银行', '北京银行', '广东南粤银行', '宁夏黄河农村商业银行', '鞍山银行','深发展', '华融湘江银行', '农商', '河南省农村信用',  '陕西信合', '大连银行', '恒丰银行', '云南省农村信用社', '兴业', '桂林银行', '广东发展银行', '广西北部湾银行', '廊坊银行', '武汉农村商业银行', '龙江银行', '吉林银行', '建设银行', '南海农村信用联社', '常熟农村商业银行', '中国农业银行', '洛阳银行', '广州银行', '吉林农信', '重庆银行', '人民银行', '广西省农村信用', '济宁银行', '江西省农村信用', '信阳银行', '中国交通银行', '交通银行', '广东省农村信用社联合社', '德州银行', '昆山农村商业银行', '无锡农村商业银行', '营口银行', '浦发', '德阳商业银行', '中行',  '绍兴银行', '晋商银行', '安阳银行', '上海农村商业银行', '齐商银行', '晋城银行', '许昌银行', '浙江泰隆商业银行', '贵州省农村信用社', '湖北银行宜昌分行', '重庆三峡银行', '上海浦东发展银行', '农业银行', '上海银行', '杭州银行', '广州农商银行', '华夏', '鄂尔多斯银行', '中国邮政储蓄银行', '苏州银行', '昆仑银行', '抚顺银行', '汉口银行', '青岛银行', '承德银行', '江苏省农村信用联合社', '东营市商业银行', '福建海峡银行', '临商银行', '浙江稠州商业银行', '成都农商银行', '农行',  '长沙银行', '齐鲁银行', '青海银行', '东莞银行', '平顶山银行', '深圳农村商业银行', '包商银行', '中国工商银行','光大', '渤海银行', '中信', '尧都农商行', '宁波银行', '湖北省农村信用社', '莱商银行', '阜新银行', '山东农信', '周口银行', '新乡银行', '赣州银行', '台州银行', '中信银行', '鄞州银行', '中国银行', '招行',  '工商银行', '成都银行', '太仓农村商业银行', '锦州银行', '邢台银行', '湖北银行', '国家开发银行', '人行', '北京农村商业银行', '吴江农商银行', '东莞农村商业银行', '河北省农村信用社', '南京银行', '重庆农村商业银行', '浙江省农村信用社联合社', '中国建设银行', '招商银行', '河北银行', '安徽省农村信用社', '常州农村信用联社', '天津农商银行', '天津银行', '金华银行', '交行', '顺德农商银行', '平安银行', '兰州银行', '浙商银行', '民生', '富滇银行', '衡水银行', '中国光大银行', '驻马店银行', '徽商银行', '长安银行', '建行',  '朝阳银行', '浙江民泰商业银行', '信合', '江苏银行', '嘉兴银行', '邮储', '江阴农村商业银行', '内蒙古银行', '石嘴山银行', '四川省农村信用', '阳泉银行', '甘肃省农村信用', '湖南省农村信用社', '丹东银行', '工行', '九江银行', '温州银行', '南昌银行', '华夏银行', '宁夏银行', '中国民生银行',  '郑州银行', '盛京银行', '上饶银行', '潍坊银行', '兴业银行', '三门峡银行','西安银行']
def getLevelWordsFromTXT(level, rdAddr):
    levelWords = []
    f = open(rdAddr, encoding='utf-8')
    for item in f:
        item=item.strip().split(',')
        for i in range(len(item)):
            if i % 2 == 1 and item[i] == 'R'+str(level):
                levelWords.append([item[i-1], 'R'+str(level)])
    f.close()
    return levelWords
sl = getLevelWordsFromTXT(23, 'data/source/dl_data_v4_21103.txt')
wrongList = []
for item in sl:
    word = item[0]
    if re.match('.*银行$', word) != None and len(word) <= 4:
        rightList.append(word)
    else:
        wrongList.append(word)
rightList, wrongList = list(set(rightList)), list(set(wrongList))
print(len(rightList), len(wrongList))

240 28206


In [135]:
wrongList = set(wrongList)
rightList = set(rightList)

In [137]:
wrongList & rightList

{'上海农村商业银行',
 '上海浦东发展银行',
 '中国人民银行',
 '中国光大银行',
 '中国农业银行',
 '中国工商银行',
 '中国建设银行',
 '中国民生银行',
 '中国邮政储蓄银行',
 '云南省农村信用社',
 '信合',
 '内蒙古银行',
 '农行',
 '北京农村商业银行',
 '华融湘江银行',
 '国家开发银行',
 '天津农商银行',
 '常熟农村商业银行',
 '平顶山银行',
 '广东南粤银行',
 '广东发展银行',
 '广州农商银行',
 '广西北部湾银行',
 '建行',
 '成都农商银行',
 '无锡农村商业银行',
 '昆山农村商业银行',
 '武汉农村商业银行',
 '河北省农村信用社',
 '浙江民泰商业银行',
 '浙江泰隆商业银行',
 '深圳农村商业银行',
 '湖南省农村信用社',
 '石嘴山银行',
 '贵州省农村信用社',
 '重庆三峡银行',
 '重庆农村商业银行',
 '顺德农商银行'}

In [109]:
'一对山商业城自贡市商业银行一对山支行' in r23_word_data_map

False

In [51]:
r3_short = readDataFiles_('/home/bangsun/research/fank/nlpAddress/codes/data/pkl/r3_short_1229.pkl')

In [53]:
r3 = []
for i in r3_short:
    r3.extend(i[0])

In [55]:
'河北' in r3

False

In [56]:
r3_short_ = list(set(readDataFiles_('./data/pkl/newPKL/r3_townShort_1229.pkl')))

In [57]:
'河北' in r3_short_

True